# Run Mike1D in notebook using dotnet interactive

## Install Mike1D from public NuGet

In [ ]:
#r "nuget:DHI.Mike.Install"

In [ ]:
#r "nuget:DHI.Mike1D.Generic"
#r "nuget:DHI.Mike1D.BoundaryModule"
#r "nuget:DHI.Mike1D.Mike1DDataAccess"
#r "nuget:DHI.Mike1D.ResultDataAccess"
#r "nuget:DHI.Mike1D.RainfallRunoffModule"

# Import

In [ ]:
using System.IO;
using System.Linq;
using System.Collections.Generic;
using DHI.Mike.Install;
using DHI.Mike1D.BoundaryModule;
using DHI.Mike1D.Generic;
using DHI.Mike1D.Mike1DDataAccess;
using DHI.Mike1D.RainfallRunoffModule;
using DHI.Mike1D.ResultDataAccess;

In [ ]:
if (MikeImport.SetupLatest())
    Console.Write(MikeImport.ActiveProduct().ToDisplayString());

## Load Mike1D example model

In [ ]:
var modelFilePath = Path.Combine("..", "mike1d-rtc-simulator-example", "MU", "RTCExampleBase.m1dx");

In [ ]:
public static Mike1DController CreateEngine(string modelFilePath) 
{
    var engineController = new Mike1DController();
    var diagnostics = new Diagnostics("notebook");
    engineController.Mike1DData = new Mike1DBridge().Open(Connection.Create(modelFilePath), diagnostics);
    return engineController;
}

## Change result file paths

In [ ]:
public static void SetResultFiles(Mike1DData mike1DData, DateTime from, DateTime to) 
{
    var currentDirectory = Directory.GetCurrentDirectory();    
    var modelFileName = Path.GetFileNameWithoutExtension(mike1DData.Connection.FilePath.ToString());

    foreach (var result in mike1DData.ResultSpecifications)
    {
        var resultFilePath = Path.Combine(currentDirectory, $"{modelFileName}_{result.ID.ToString()}.res1d");
        result.Connection.FilePath = new FilePath(resultFilePath);
        result.StartTime = from;
        result.EndTime = to;
    }
}

## Prepare and run simulation

In [ ]:
public static void Prepare(IMike1DController engine, IDiagnostics diagnostics = null)
{
    diagnostics = diagnostics ?? new Diagnostics();

    if (diagnostics.ErrorCountRecursive > 0)
        throw new Exception("Loading errors, aborting. " + diagnostics.ErrorsRecursive[0].Message);

    IDiagnostics validation = engine.Validate();

    if (validation.ErrorCountRecursive > 0)
        throw new Exception("Validation errors, aborting. " + validation.ErrorsRecursive[0].Message);

    engine.Initialize(diagnostics);

    if (diagnostics.ErrorCountRecursive > 0)
        throw new Exception("Initialization errors, aborting. " + diagnostics.ErrorsRecursive[0].Message);

    engine.Prepare();
}

In [ ]:
public static void SetTimePeriod(IMike1DController engine, DateTime from, DateTime to)
{
    SetResultFiles(engine.Mike1DData, from, to);    
    engine.Mike1DData.SimulationStart = from;
    engine.Mike1DData.SimulationEnd = to;
}

In [ ]:
var engineController = CreateEngine(modelFilePath);
// SetTimePeriod(engineController, new DateTime(1953, 05, 16, 0, 0, 0), new DateTime(1953, 05, 16, 1, 0, 0))
Prepare(engineController);

In [ ]:
engineController.CurrentTime

In [ ]:
engineController.RunUntil(engineController.CurrentTime.AddMinutes(5))

In [ ]:
engineController.Finish()